In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import h5py
import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation
from keras import optimizers, layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, coverage_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer

In [3]:
import pandas as pd
whole = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/ac.csv')
train, test = train_test_split (whole,test_size=0.2)
train.to_csv (path_or_buf = 'C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv')
test.to_csv (path_or_buf = 'C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv')

In [ ]:
# Training cost (IM1, IM2, IM3, OS1), (11,12,13,7)
# ivis=46, iev=47, ilc=50, iqt=51, icost=52, icd=54, ipro=55, iothers=57
# CS=58, BT=59, per=60, PT=61, Location=62, yr=63, contract=64

In [21]:
#train set
x_o_train = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    delimiter = ',', usecols = (11,12,13,7),skip_header =2)
y_o_train = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    delimiter = ',', usecols = (46,47,50,51,52,54,55,57),skip_header =2)
CS_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["CS_cat"], header = 1)
BT_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["BT_cat"], header = 1)
per_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["per_cat"], header = 1)
PT_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["PT_cat"], header = 1)
L_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["Location"], header = 1)
yr_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["yr_cat"], header = 1)
contract_train = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/train_cat_ac.csv', 
                    names = ["contract"], header = 1)
#CS_train = CS_train [2:,:]
#BT_train = BT_train [2:,:]
#per_train = per_train [2:,:]
#PT_train = PT_train [2:,:]
#L_train = L_train [2:,:]
#yr_train = yr_train [2:,:]
#contract_train = contract_train [2:,:]

In [25]:
#test set
x_o_test = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    delimiter = ',', usecols = (11,12,13,7),skip_header =1)
y_o_test = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    delimiter = ',', usecols = (46,47,50,51,52,54,55,57),skip_header =1)
CS_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["CS_cat"], header =0)
BT_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["BT_cat"], header =0)
per_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["per_cat"], header =0)
PT_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["PT_cat"], header = 0)
L_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["Location"], header =0)
yr_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["yr_cat"], header =0)
contract_test = pd.read_csv ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Feature_selected/test_cat_ac.csv', 
                    names = ["contract"], header =0)

(82, 4)
(82, 1)


In [30]:
lb = LabelBinarizer (neg_label=-1, pos_label=1)
#CS
lb.fit (CS_train)
CS_train = lb.transform (CS_train)
CS_test = lb.transform (CS_test)

#BT
lb.fit (BT_train)
BT_train = lb.transform (BT_train)
BT_test = lb.transform (BT_test)

#per
lb.fit (per_train)
per_train = lb.transform (per_train)
per_test = lb.transform (per_test)


#PT
lb.fit (PT_train)
PT_train = lb.transform (PT_train)
PT_test = lb.transform (PT_test)


#Location
lb.fit (L_train)
L_train = lb.transform (L_train)
L_test = lb.transform (L_test)


#yr
lb.fit (yr_train)
yr_train = lb.transform (yr_train)
yr_test = lb.transform (yr_test)


#contract
lb.fit (contract_train)
contract_train = lb.transform (contract_train)
contract_test = lb.transform (contract_test)

In [35]:
scaler = StandardScaler ()
scaler.fit (x_o_train)
x_o_train = scaler.transform (x_o_train)
x_o_test = scaler.transform (x_o_test)

x_train_split = np.hstack ((x_o_train, CS_train, BT_train, per_train, PT_train, L_train, yr_train, contract_train))
x_test = np.hstack ((x_o_test, CS_test, BT_test, per_test, PT_test, L_test, yr_test, contract_test))

x_train, x_cv, y_train, y_cv = train_test_split (x_train_split,y_o_train,test_size=0.3)

In [36]:
print (x_train.shape)
print (y_train.shape)
print (x_cv.shape)
#print (CS_train.shape)
#print (CS_train)

(226, 11)
(226, 8)
(97, 11)


In [39]:
y_test = y_o_test;
Neuron = 13;
num_classes = 8;
for Neuron in range (10,20):
    model = Sequential ()
    model.add (Dense (units = Neuron, input_dim = 11))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (layers.Activation ('tanh'))
    model.add (Dense (units = Neuron))
    model.add (Dense (num_classes))
    print (model.summary())

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
    model.compile (loss = 'binary_crossentropy', optimizer = 'adam', metrics=['binary_accuracy'])
    print(model.fit(x_train, y_train, batch_size=80, verbose=1, validation_data=(x_cv, y_cv)))
    print(model.evaluate (x_test, y_test, batch_size = 50, verbose =1))
#help (model.evaluate)
#this is the output units
    y_pred_train = model.predict (x_train)
    y_pred_cv = model.predict (x_cv)
    y_pred_test = model.predict (x_test)
    y_pred_whole = np.vstack ((y_pred_train, y_pred_cv)).astype(np.float32)
    
    #creating 100 random numbers
    y_pred = y_pred_whole
    f = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Keras/f_random.csv', 
                    delimiter = ',').reshape(100,1)
    def t(x):
        return f.item (x)

    def k (x):
        k = np.zeros ((323,8));
        i = 0;
        j = 0;
        for i in range (323):
            for j in range (8):
                if y_pred.item((i,j))<=t(x):
                    np.put(k,[8*i+j],[1]);
        return k

    g = np.zeros((323,1))
    for x in range (100):
        m = np.sum(k(x), axis = 1).reshape(323,1);
    #print (m.shape)
        g = np.hstack((g,m));
    g = np.delete(g,0,1)


    i = 0
    j = 0
    m = np.zeros((323,100))
    for i in range (323):
        min =100
        for j in range (100):
            if g.item ((i,j))<=min:
                min = g.item((i,j))
        for j in range (100):
            if g.item ((i,j))==min:
                np.put(m,[100*i+j],[1])
            
    i = 0
    j = 0
    w = np.zeros((323,100))
    for i in range (0,323):
        for j in range (0, 100):
            w_ = m.item((i,j))*t(j)
            np.put(w,[100*i+j],[w_])

        
    cnt = np.count_nonzero(w, axis=1).reshape(323,1)
    re_cnt = cnt[::-1]
    w_total = np.sum(w, axis = 1).reshape(323,1)
    w_avg = w_total/re_cnt
    A = np.hstack([y_pred, np.ones((323,1))])
    w1,w2,w3,w4,w5,w6,w7,w8,b = np.linalg.lstsq(A,w_avg)[0]
    print (w1,w2,w3,w4,w5,w6,w7,w8,b)
    
    
    def y_label (y_pred):
    
        I = y_pred.shape[0];
        J = y_pred.shape[1];
    
        x1_Y = y_pred [:,0].reshape(I,1);
        x2_Y = y_pred [:,1].reshape(I,1);
        x3_Y = y_pred [:,2].reshape(I,1);
        x4_Y = y_pred [:,3].reshape(I,1);
        x5_Y = y_pred [:,4].reshape(I,1);
        x6_Y = y_pred [:,5].reshape(I,1);
        x7_Y = y_pred [:,6].reshape(I,1);
        x8_Y = y_pred [:,7].reshape(I,1);
        #x9_Y = y_pred [:,8].reshape(I,1);
        #x10_Y = y_pred [:,9].reshape(I,1);
    
        c_x = y_pred.reshape(I,J);
        y_thre = np.zeros((I,1));
        y_label = np.zeros((I,J));
    
        i = 0;
        j = 0;
    
        for i in range (I):
            x1 = x1_Y.item(i,0);
            x2 = x2_Y.item(i,0);
            x3 = x3_Y.item(i,0);
            x4 = x4_Y.item(i,0);
            x5 = x5_Y.item(i,0);
            x6 = x6_Y.item(i,0);
            x7 = x7_Y.item(i,0);
            x8 = x8_Y.item(i,0);
            #x9 = x9_Y.item(i,0);
            #x10 = x10_Y.item(i,0);
            
            y_tx = w1*x1+w2*x2+w3*x3+w4*x4+w5*x5+w6*x6+w7*x7+w8*x8+b#w9*x9+b+w10*x10;
            np.put (y_thre, [i], [y_tx]);
        
        for i in range (I):
            for j in range (J):
                threshold = y_thre.item(i,0);
                if c_x.item(i,j) >= threshold:
                    np.put(y_label, [J*i+j], [1])
    
        return (y_label)    
    
    
    y_train_label = y_label(y_pred_train)
    y_cv_label = y_label(y_pred_cv)
    y_test_label = y_label(y_pred_test)
    y_t_label = np.vstack((y_train_label,y_cv_label))
    y_t = np.vstack((y_train,y_cv))

    print('hamming_loss_train',hamming_loss(y_train, y_train_label))
    print('hamming_loss_cv',hamming_loss(y_cv, y_cv_label))
    print('hamming_loss_test',hamming_loss(y_test, y_test_label))
    print('hamming_loss_traindataset',hamming_loss(y_t, y_t_label))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 10)                120       
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_13 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                110       
__________

Train on 226 samples, validate on 97 samples
Epoch 1/10
226/226 [==============================] - ETA: 0s - loss: 3.4536 - binary_accuracy: 0.6405 - val_loss: 3.3606 - val_binary_accuracy: 0.6959
Epoch 2/10
226/226 [==============================] - ETA: 0s - loss: 3.2150 - binary_accuracy: 0.6936 - val_loss: 2.9553 - val_binary_accuracy: 0.7010
Epoch 3/10
226/226 [==============================] - ETA: 0s - loss: 2.8293 - binary_accuracy: 0.6952 - val_loss: 2.6641 - val_binary_accuracy: 0.6985
Epoch 4/10
226/226 [==============================] - ETA: 0s - loss: 2.6702 - binary_accuracy: 0.6869 - val_loss: 2.5567 - val_binary_accuracy: 0.6778
Epoch 5/10
226/226 [==============================] - ETA: 0s - loss: 2.4985 - binary_accuracy: 0.6692 - val_loss: 2.3805 - val_binary_accuracy: 0.6508
Epoch 6/10
226/226 [==============================] - ETA: 0s - loss: 2.1857 - binary_accuracy: 0.6554 - val_loss: 1.8109 - val_binary_accuracy: 0.6340
Epoch 7/10
226/226 [=======================

226/226 [==============================] - ETA: 0s - loss: 2.2102 - binary_accuracy: 0.7035 - val_loss: 1.8079 - val_binary_accuracy: 0.7101
Epoch 4/10
226/226 [==============================] - ETA: 0s - loss: 1.7913 - binary_accuracy: 0.7041 - val_loss: 1.3306 - val_binary_accuracy: 0.7152
Epoch 5/10
226/226 [==============================] - ETA: 0s - loss: 1.4523 - binary_accuracy: 0.7030 - val_loss: 0.9818 - val_binary_accuracy: 0.7049
Epoch 6/10
226/226 [==============================] - ETA: 0s - loss: 1.0885 - binary_accuracy: 0.6947 - val_loss: 0.7538 - val_binary_accuracy: 0.7036
Epoch 7/10
226/226 [==============================] - ETA: 0s - loss: 0.8645 - binary_accuracy: 0.7035 - val_loss: 0.6393 - val_binary_accuracy: 0.7126
Epoch 8/10
226/226 [==============================] - ETA: 0s - loss: 0.7498 - binary_accuracy: 0.7030 - val_loss: 0.6137 - val_binary_accuracy: 0.7126
Epoch 9/10
226/226 [==============================] - ETA: 0s - loss: 0.7078 - binary_accuracy: 0.7

226/226 [==============================] - ETA: 0s - loss: 1.8840 - binary_accuracy: 0.6643 - val_loss: 1.6678 - val_binary_accuracy: 0.6559
Epoch 6/10
226/226 [==============================] - ETA: 0s - loss: 1.7156 - binary_accuracy: 0.6593 - val_loss: 1.6142 - val_binary_accuracy: 0.6508
Epoch 7/10
226/226 [==============================] - ETA: 0s - loss: 1.5893 - binary_accuracy: 0.6521 - val_loss: 1.5015 - val_binary_accuracy: 0.6456
Epoch 8/10
226/226 [==============================] - ETA: 0s - loss: 1.5062 - binary_accuracy: 0.6482 - val_loss: 1.4740 - val_binary_accuracy: 0.6482
Epoch 9/10
226/226 [==============================] - ETA: 0s - loss: 1.4423 - binary_accuracy: 0.6460 - val_loss: 1.4387 - val_binary_accuracy: 0.6495
Epoch 10/10
82/82 [==============================] - ETA: 0s
[1.1731089615240329, 0.67073171894724781]
[ 1.16704079] [-1.05382788] [ 0.03280007] [-5.90706639] [ 1.52116965] [-1.07878117] [-1.4893756] [-2.98675699] [ 0.09791884]
hamming_loss_train 0.39

226/226 [==============================] - ETA: 0s - loss: 1.8610 - binary_accuracy: 0.6781 - val_loss: 1.7339 - val_binary_accuracy: 0.7062
Epoch 8/10
226/226 [==============================] - ETA: 0s - loss: 1.8648 - binary_accuracy: 0.6875 - val_loss: 1.7454 - val_binary_accuracy: 0.7126
Epoch 9/10
226/226 [==============================] - ETA: 0s - loss: 1.8677 - binary_accuracy: 0.6925 - val_loss: 1.7434 - val_binary_accuracy: 0.7126
Epoch 10/10
82/82 [==============================] - ETA: 0s
[1.6556191473472408, 0.70121951219512191]
[ 0.29410637] [ 4.1823829] [ 2.50074849] [ 0.39934441] [ 6.42058063] [-0.91986694] [ 0.88261898] [-0.759057] [ 0.12140522]
hamming_loss_train 0.3329646017699115
hamming_loss_cv 0.327319587628866
hamming_loss_test 0.3475609756097561
hamming_loss_traindataset 0.33126934984520123
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 19)   

In [41]:
Neuron = 15;
num_classes = 8;
model = Sequential ()
model.add (Dense (units = Neuron, input_dim = 11))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = Neuron))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = Neuron))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = Neuron))
model.add (Dense(num_classes))
#print (model.summary())

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
model.compile (loss = 'binary_crossentropy', optimizer = 'adam', metrics=['binary_accuracy'])
print(model.fit(x_train, y_train,batch_size=200, verbose=1, validation_data=(x_cv, y_cv)))
print(model.evaluate (x_test, y_test, batch_size = 50, verbose =1))
#help (model.evaluate)
#this is the output units
y_pred_train = model.predict (x_train)
y_pred_cv = model.predict (x_cv)
y_pred_test = model.predict (x_test)
y_pred_whole = np.vstack ((y_pred_train, y_pred_cv)).astype(np.float32)
    
#creating 100 random numbers
y_pred = y_pred_whole
f = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Jupyter/Keras/f_random.csv', 
                    delimiter = ',').reshape(100,1)
def t(x):
    return f.item (x)

def k (x):
    k = np.zeros ((323,8));
    i = 0;
    j = 0;
    for i in range (323):
        for j in range (8):
            if y_pred.item((i,j))<=t(x):
                np.put(k,[8*i+j],[1]);
    return k

g = np.zeros((323,1))
for x in range (100):
    m = np.sum(k(x), axis = 1).reshape(323,1);
    #print (m.shape)
    g = np.hstack((g,m));
g = np.delete(g,0,1)


i = 0
j = 0
m = np.zeros((323,100))
for i in range (323):
    min =100
    for j in range (100):
        if g.item ((i,j))<=min:
            min = g.item((i,j))
    for j in range (100):
        if g.item ((i,j))==min:
            np.put(m,[100*i+j],[1])
            
i = 0
j = 0
w = np.zeros((323,100))
for i in range (0,323):
    for j in range (0, 100):
        w_ = m.item((i,j))*t(j)
        np.put(w,[100*i+j],[w_])

        
cnt = np.count_nonzero(w, axis=1).reshape(323,1)
re_cnt = cnt[::-1]
w_total = np.sum(w, axis = 1).reshape(323,1)
w_avg = w_total/re_cnt
A = np.hstack([y_pred, np.ones((323,1))])
w1,w2,w3,w4,w5,w6,w7,w8,b = np.linalg.lstsq(A,w_avg)[0]
print (w1,w2,w3,w4,w5,w6,w7,w8,b)
    
    
def y_label (y_pred):
    
    I = y_pred.shape[0];
    J = y_pred.shape[1];
    
    x1_Y = y_pred [:,0].reshape(I,1);
    x2_Y = y_pred [:,1].reshape(I,1);
    x3_Y = y_pred [:,2].reshape(I,1);
    x4_Y = y_pred [:,3].reshape(I,1);
    x5_Y = y_pred [:,4].reshape(I,1);
    x6_Y = y_pred [:,5].reshape(I,1);
    x7_Y = y_pred [:,6].reshape(I,1);
    x8_Y = y_pred [:,7].reshape(I,1);
        #x9_Y = y_pred [:,8].reshape(I,1);
        #x10_Y = y_pred [:,9].reshape(I,1);
    
    c_x = y_pred.reshape(I,J);
    y_thre = np.zeros((I,1));
    y_label = np.zeros((I,J));
    
    i = 0;
    j = 0;
    
    for i in range (I):
        x1 = x1_Y.item(i,0);
        x2 = x2_Y.item(i,0);
        x3 = x3_Y.item(i,0);
        x4 = x4_Y.item(i,0);
        x5 = x5_Y.item(i,0);
        x6 = x6_Y.item(i,0);
        x7 = x7_Y.item(i,0);
        x8 = x8_Y.item(i,0);
            #x9 = x9_Y.item(i,0);
            #x10 = x10_Y.item(i,0);
            
        y_tx = w1*x1+w2*x2+w3*x3+w4*x4+w5*x5+w6*x6+w7*x7+w8*x8+b#w9*x9+b+w10*x10;
        np.put (y_thre, [i], [y_tx]);
        
    for i in range (I):
        for j in range (J):
            threshold = y_thre.item(i,0);
            if c_x.item(i,j) >= threshold:
                np.put(y_label, [J*i+j], [1])
    
    return (y_label)    
    
    
y_train_label = y_label(y_pred_train)
y_cv_label = y_label(y_pred_cv)
y_test_label = y_label(y_pred_test)
y_t_label = np.vstack((y_train_label,y_cv_label))
y_t = np.vstack((y_train,y_cv))

print('hamming_loss_train',hamming_loss(y_train, y_train_label))
print('hamming_loss_cv',hamming_loss(y_cv, y_cv_label))
print('hamming_loss_test',hamming_loss(y_test, y_test_label))
print('hamming_loss_traindataset',hamming_loss(y_t, y_t_label))

Train on 226 samples, validate on 97 samples
Epoch 1/10
226/226 [==============================] - ETA: 0s - loss: 2.3333 - binary_accuracy: 0.6538 - val_loss: 2.0051 - val_binary_accuracy: 0.6534
Epoch 2/10
226/226 [==============================] - ETA: 0s - loss: 2.0015 - binary_accuracy: 0.6632 - val_loss: 1.7838 - val_binary_accuracy: 0.6611
Epoch 3/10
226/226 [==============================] - ETA: 0s - loss: 1.7868 - binary_accuracy: 0.6704 - val_loss: 1.6402 - val_binary_accuracy: 0.6791
Epoch 4/10
226/226 [==============================] - ETA: 0s - loss: 1.5706 - binary_accuracy: 0.6759 - val_loss: 1.5486 - val_binary_accuracy: 0.6765
Epoch 5/10
226/226 [==============================] - ETA: 0s - loss: 1.4341 - binary_accuracy: 0.6798 - val_loss: 1.4912 - val_binary_accuracy: 0.6753
Epoch 6/10
226/226 [==============================] - ETA: 0s - loss: 1.3810 - binary_accuracy: 0.6759 - val_loss: 1.4380 - val_binary_accuracy: 0.6843
Epoch 7/10
226/226 [=======================

In [42]:
#save model to json, weights to HDF5
model_json = model.to_json ()
with open ('trainC.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('trainC.h5')
print ('Saved model to disk')

Saved model to disk
